In [77]:
import pandas as pd
import re
from timeit import default_timer as timer
from joblib import Parallel, delayed
from joblib import effective_n_jobs
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import random
from scipy.stats.mstats import winsorize
from IPython.display import Latex
from IPython.display import Markdown, display
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

### Examples - Working with Safegraph data

**Timing code**

I use the default timer from the timeit library to estimate running time. E.g., function below took ~14 seconds

In [78]:
def dummy(i):
    df = pd.read_csv('safegraph_data.csv')
    row = df.iloc[[1]]

In [80]:
start = timer()
for i in range(0,3000):
    dummy(i)
timer()-start

8.489704082999197

**Parallelizing**

One way to speed up code is to make use of all CPUs. Multiple libraries in Python allow you to parallel process, below is an example using joblib parallel. Setting n_jobs=-1 will use all available CPUs. If you want to continue using your computer to work on something else, consider n_jobs=-2 or -3.

In [81]:
start = timer()
Parallel(n_jobs=-1)(delayed(dummy)(i) for i in range(0,3000))
end = timer()
end-start

3.7345100000002276

**Vectorization > Loops**

Vectorization is the technique of implementing (NumPy) array operations on a dataset. In the background, it applies the operations to all the elements of an array or series in one go (unlike a ‘for’ loop that manipulates one row at a time).

In the dataframe below, let's say we wanted to multiply raw visit counts by the median dwell time (as a fraction of an hour).

In [83]:
df = pd.read_csv('safegraph_data.csv')
df.head(3)

,V1,placekey,parent_placekey,location_name,city,poi_cbg,visitor_home_cbgs,visitor_home_aggregation,bucketed_dwell_times,naics_code,polygon_wkt,wkt_area_sq_meters,geometry_type,date_range_start,date_range_end,opened_on,closed_on,iso_country_code,visits_by_day,popularity_by_hour,raw_visit_counts,normalized_visits_by_state_scaling,median_dwell,region,top_category,sub_category,scaling_factor
0,63283,zzy-222@5vh-b7t-ckf,NaN,The Dish Stanford Foothills,Stanford,60855115004,"{""""060855115005"""":12,""""060855115004"""":9,""""0608...","{""""06085511500"""":29,""""06085511100"""":14,""""06085...","{""""<5"""":80,""""5-10"""":461,""""11-20"""":217,""""21-60""...",712190,"POLYGON ((-122.16060739633784 37.410283, -122....",1236,POLYGON,2020-10-01T07:00:00Z,2020-11-01T07:00:00Z,NaN,NaN,US,"[77,64,48,44,79,92,71,113,92,60,51,76,88,91,70...","[102,96,78,78,83,150,234,316,476,554,582,612,5...",2252,47345.778627,50.0,CA,"Museums, Historical Sites, and Similar Institu...",Nature Parks and Other Similar Institutions,21.02388
1,414659,222-222@5vh-b7v-kcq,NaN,Terman Park,Stanford,60855116081,"{""""060855031132"""":4}","{""""06081610400"""":4}","{""""<5"""":0,""""5-10"""":3,""""11-20"""":2,""""21-60"""":6,""...",712190,POLYGON ((-122.17320150599994 37.4259167750000...,270,POLYGON,2020-10-01T07:00:00Z,2020-11-01T07:00:00Z,NaN,NaN,US,"[0,0,0,0,3,0,2,0,0,0,0,1,0,1,0,1,0,0,0,1,1,0,0...","[0,0,0,0,1,1,2,2,3,7,7,6,6,5,3,2,0,1,1,0,1,0,0,0]",18,378.429847,40.5,CA,"Museums, Historical Sites, and Similar Institu...",Nature Parks and Other Similar Institutions,21.02388
2,452914,zzy-223@5vh-b7v-j5f,NaN,Rodin Sculpture Garden,Stanford,60855116081,"{""""060855130002"""":7,""""340030472004"""":5,""""06001...","{""""06085513000"""":12,""""06013303102"""":7,""""060855...","{""""<5"""":13,""""5-10"""":89,""""11-20"""":58,""""21-60"""":...",712110,"POLYGON ((-122.17062963020015 37.4322972, -122...",1236,POLYGON,2020-10-01T07:00:00Z,2020-11-01T07:00:00Z,NaN,NaN,US,"[16,20,12,13,23,22,22,20,16,11,17,21,20,16,11,...","[11,10,15,7,6,17,22,25,47,63,84,86,79,72,79,84...",499,10490.916312,40.0,CA,"Museums, Historical Sites, and Similar Institu...",Museums,21.02388


One instinct might be to do this row by row:

In [64]:
start = timer()
# Iterating through DataFrame using iterrows
for idx, row in df.iterrows():
    # creating a new column 
    df.at[idx,'visits_new'] = row["raw_visit_counts"] * (row["median_dwell"]/60)  
rt1 = timer() - start
print(rt1)

0.01189129199974559


In [62]:
start = timer()
df["visits_new"] = df["raw_visit_counts"] * df["median_dwell"]/60
rt2 = timer() - start
print(rt2)

0.0017731660000208649


We see that iterating through dataframe is almost 7 times slower. This makes a huge difference when working with millions of rows.

**Matrix operations**

In a similar vein, matrix operations using Numpy are almost always the fastest way of working with the data. In the data below, let's say we wanted to estimate the number of visitors per hour per day. Monthly data on a POI contains information on the number of visits to a POI on a given day. As an example, below is an array *d* showing number of visitors to the dish in October 2020:

In [70]:
def get_arrays(r):
    r = [re.sub('[\[\]]','',i).split(',') for i in r]
    r = [[int(j) for j in i] for i in r]
    r = np.array(r)
    return(r)

dish = df.head(1)
dish

,V1,placekey,parent_placekey,location_name,city,poi_cbg,visitor_home_cbgs,visitor_home_aggregation,bucketed_dwell_times,naics_code,...,visits_by_day,popularity_by_hour,raw_visit_counts,normalized_visits_by_state_scaling,median_dwell,region,top_category,sub_category,scaling_factor,visits_new
0,63283,zzy-222@5vh-b7t-ckf,NaN,The Dish Stanford Foothills,Stanford,60855115004,"{""""060855115005"""":12,""""060855115004"""":9,""""0608...","{""""06085511500"""":29,""""06085511100"""":14,""""06085...","{""""<5"""":80,""""5-10"""":461,""""11-20"""":217,""""21-60""...",712190,...,"[77,64,48,44,79,92,71,113,92,60,51,76,88,91,70...","[102,96,78,78,83,150,234,316,476,554,582,612,5...",2252,47345.778627,50.0,CA,"Museums, Historical Sites, and Similar Institu...",Nature Parks and Other Similar Institutions,21.02388,1876.666667


In [152]:
day_visits = get_arrays(dish['visits_by_day'])
day_visits

array([[ 77,  64,  48,  44,  79,  92,  71, 113,  92,  60,  51,  76,  88,
         91,  70,  77,  49,  34,  76, 104,  82,  87,  91,  42,  36,  87,
         76,  89,  88,  76,  42]])

However, it is unlikely that the average number of people at 2 am was the same as the people there at 1 pm. To capture changes in visitors throughout the day, we can use data on the popularity of the POI by hour. Showing this data for the dish:

In [153]:
hour_visits = get_arrays(dish['popularity_by_hour'])
hour_visits

array([[102,  96,  78,  78,  83, 150, 234, 316, 476, 554, 582, 612, 587,
        527, 470, 435, 358, 253, 259, 190, 168, 142, 121, 112]])

We rewrite this by dividing each element by the sum of the row. E.g., for row above:

$$
h = \frac{1}{6983}h
$$

In [154]:
hour_visits = hour_visits/hour_visits.sum(axis=1)[:,None]
hour_visits

array([[0.0146069 , 0.01374767, 0.01116998, 0.01116998, 0.01188601,
        0.02148074, 0.03350995, 0.04525276, 0.06816554, 0.07933553,
        0.08334527, 0.08764141, 0.08406129, 0.075469  , 0.06730632,
        0.06229414, 0.05126736, 0.03623085, 0.03709008, 0.02720894,
        0.02405843, 0.0203351 , 0.0173278 , 0.01603895]])

Then, we want to multiply this array by the daily visitor array. One possibility would be to loop through array *d* and multiply this number by *h*. 

In [189]:
start = timer()
visits_hr_day1 = np.zeros([31,24])
for i in range(0,len(day_visits[0])):
    v = day_visits[0][i]
    curr = hour_visits*v
    visits_hr_day1 = np.append(visits_hr_day1,curr)
t1 = timer()-start
print(t1)

0.0018409999993309611


While this would work, it would be much faster to compute this in a single operation by using dot products. We can estimate the number of visits X on day d at hour h as follows:

$$
\begin{pmatrix}
  x_{1,1} & x_{1,2} & \cdots & x_{1,24} \\
  x_{2,1} & x_{2,2} & \cdots & x_{2,n} \\
  \vdots  & \vdots  & \ddots & \vdots  \\
  x_{30,1} & x_{30,2} & \cdots & x_{30,24} 
\end{pmatrix}
=
\begin{pmatrix}
  d_{1} \\
  d_{2} \\
  \vdots \\
  d_{30}
\end{pmatrix}
\cdot
\left(
\frac{1}{\sum_{n=1}^{n=24}h_n}
\begin{pmatrix}
  h_{1} \\ 
  h_{2} \\ 
  \vdots \\
  h_{24}
\end{pmatrix}
\right)^\intercal
$$

In [190]:
start = timer()
d1 = np.asmatrix(day_visits[0]).reshape(-1,1)
h1 = np.asmatrix(hour_visits[0])
visits_hr_day2 = np.dot(d1,h1)
t2 = timer()-start
print(t2)

0.0006882079997012625


In [191]:
t1/t2

2.675063353128855